# Import all the necessary packages

In [1]:
import csv
from enum import IntEnum
import time
import jsonrpclib
import subprocess
from subprocess import PIPE, Popen
from threading  import Thread
import sys
import logging
import re
from collections import OrderedDict
import threading
import queue

import PySimpleGUI as sg

from gym import Env, error, spaces, utils
from stable_baselines3 import DQN, PPO, A2C, TD3, SAC
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.callbacks import BaseCallback, CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common import results_plotter
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import multivariate_normal

import os
import requests
import shutil
import tempfile
import xml.etree.ElementTree as ET
from io import StringIO, BytesIO

import cv2
import numpy as np
import torch
from PIL import Image
from IPython.display import clear_output
import gym
from cv2 import QRCodeDetector
from pyzbar import pyzbar

import olympe
from olympe.messages.ardrone3.Piloting import TakeOff, Landing, moveBy, PCMD, moveTo
from olympe.messages.ardrone3.PilotingState import FlyingStateChanged, PositionChanged, GpsLocationChanged, moveToChanged
from olympe.enums.ardrone3.PilotingState import FlyingStateChanged_State as FlyingState
from olympe.messages.ardrone3.GPSSettingsState import GPSFixStateChanged, HomeChanged
from olympe.messages.common.CommonState import BatteryStateChanged
from olympe.messages.gimbal import set_target, attitude
from olympe.messages.camera import (
    set_camera_mode,
    set_photo_mode,
    take_photo,
    photo_progress,
)
from olympe.media import (
    media_created,
    resource_created,
    media_removed,
    resource_removed,
    resource_downloaded,
    indexing_state,
    delete_media,
    download_media,
    download_media_thumbnail,
    MediaEvent,
)

from pynput.keyboard import Listener, Key, KeyCode
from collections import defaultdict

olympe.log.update_config({
    "loggers": {
        "olympe": {
                "handlers": []
            }
        },
        "ulog": {
            "level": "OFF",
            "handlers": [],
        }
})

# Define the constants

In [2]:
DRONE_IP = os.environ.get("DRONE_IP", "10.202.0.1")
DRONE_MEDIA_PORT = os.environ.get("DRONE_MEDIA_PORT", "80")
DRONE_RTSP_PORT = os.environ.get("DRONE_RTSP_PORT")

ANAFI_URL = "http://{}/".format(DRONE_IP)
ANAFI_MEDIA_API_URL = ANAFI_URL + "api/v1/media/medias/"

In [3]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [4]:
formatter = logging.Formatter('%(created)f,%(message)s')

log_dir = "deployment_logs/"
os.makedirs(log_dir, exist_ok=True)

timestr = time.strftime("%Y%m%d-%H%M%S")
filename = timestr + ".log"
abs_filename = os.path.join(log_dir, filename)
file_handler = logging.FileHandler(abs_filename, mode='w')

# with open(abs_filename, 'w', encoding='utf-8') as f:
#     f.write("timestamp,latitude,longitude,altitude,event,current_cell,next_cell,battery\n")
    
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

In [5]:
logger.addHandler(file_handler)

# Define the classes

## Action

In [6]:
class Action:
    def __init__(self, drone):
        self.drone = drone
        self.home = self.drone.get_state(GpsLocationChanged)
        
        self.current_cell = self._get_cell(13)
        self.invalid_left_cells = [1, 6, 11, 16, 21]
        self.invalid_forward_cells = [1, 2, 3, 4, 5]
        self.invalid_right_cells = [5, 10, 15, 20, 25]
        self.invalid_backward_cells = [21, 22, 23, 24, 25]
        
        self.Move = IntEnum(
            'MOVE',
            'FORWARD BACKWARD LEFT RIGHT FORWARD_LEFT FORWARD_RIGHT BACKWARD_LEFT BACKWARD_RIGHT HOVER',
            start=0
        )
        
    def take_action(self, action):
        next_cell_id = self._get_next_cell_id(action)
        next_cell = self._get_cell(next_cell_id)
        
        old_cell_id, new_cell_id = self.current_cell["id"], next_cell["id"]
        if old_cell_id == new_cell_id: 
            return old_cell_id, new_cell_id, self._get_action_name(action)
        
        self._move_to_cell(next_cell)
        
        self.current_cell = next_cell
        
        return old_cell_id, new_cell_id, self._get_action_name(action)
        
    def reset(self):
        next_cell = self._get_cell(13)
        self._move_to_cell(next_cell)
        
        old_cell_id, new_cell_id = self.current_cell["id"], next_cell["id"]
        self.current_cell = next_cell
        
        return old_cell_id, new_cell_id
    
    def _get_cell(self, cell_id):
        return self._cell_coords[cell_id - 1]
    
    def _get_action_name(self, action):
        direction = str(self.Move(action)).split(".")[1]
        code = ""
        for i in direction.split("_"):
            if i != "":
                code += i[0].upper()
        
        return code
#         direction = str(self.Move(action)).split(".")[1].capitalize()
#         return "Moving " + direction if "hover" not in direction.lower() else "Hovering"
    
    def _move_to_cell(self, next_cell):        
        self.drone(
            moveTo(next_cell["latitude"],  next_cell["longitude"], next_cell["altitude"], "HEADING_DURING", 90.0)
            >> moveToChanged(status="DONE", _timeout=15)
        ).wait()
    
    def _get_next_cell_id(self, action):
        if action == self.Move.HOVER:
            return self.current_cell["id"]
        elif action == self.Move.LEFT:
            if self.current_cell["id"] in self.invalid_left_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] - 1
        elif action == self.Move.RIGHT:
            if self.current_cell["id"] in self.invalid_right_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] + 1
        elif action == self.Move.FORWARD:
            if self.current_cell["id"] in self.invalid_forward_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] - 5
        elif action == self.Move.BACKWARD:
            if self.current_cell["id"] in self.invalid_backward_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] + 5
        elif action == self.Move.FORWARD_RIGHT:
            if self.current_cell["id"] in self.invalid_forward_cells + self.invalid_right_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] - 4
        elif action == self.Move.FORWARD_LEFT:
            if self.current_cell["id"] in self.invalid_forward_cells + self.invalid_left_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] - 6
        elif action == self.Move.BACKWARD_RIGHT:
            if self.current_cell["id"] in self.invalid_backward_cells + self.invalid_right_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] + 6
        elif action == self.Move.BACKWARD_LEFT:
            if self.current_cell["id"] in self.invalid_backward_cells + self.invalid_left_cells:
                return self.current_cell["id"]
            next_cell_id = self.current_cell["id"] + 4
            
        return next_cell_id
    
    @property
    def _cell_coords(self):
        altitude = 2.5
        dlong = 6.8e-5 * (2/5) # in degrees == 5 meters along x-axis (forward[+]-backward[-])
        dlat = 7.2e-5 * (3.2/8) # in degrees == 8 meters along y-axis (left[+]-right[-])
        
        home_lat = self.home["latitude"]
        home_long = self.home["longitude"]
        
        return [
            # cell no. 1
            OrderedDict([('id', 1),
                         ('latitude', home_lat + 2 * dlat),
                         ('longitude', home_long + 2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 2
            OrderedDict([('id', 2),
                         ('latitude', home_lat + 1 * dlat),
                         ('longitude', home_long + 2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 3
            OrderedDict([('id', 3),
                         ('latitude', home_lat + 0 * dlat),
                         ('longitude', home_long + 2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 4
            OrderedDict([('id', 4),
                         ('latitude', home_lat + -1 * dlat),
                         ('longitude', home_long + 2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 5
            OrderedDict([('id', 5),
                         ('latitude', home_lat + -2 * dlat),
                         ('longitude', home_long + 2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 6
            OrderedDict([('id', 6),
                         ('latitude', home_lat + 2 * dlat),
                         ('longitude', home_long + 1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 7
            OrderedDict([('id', 7),
                         ('latitude', home_lat + 1 * dlat),
                         ('longitude', home_long + 1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 8
            OrderedDict([('id', 8),
                         ('latitude', home_lat + 0 * dlat),
                         ('longitude', home_long + 1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 9
            OrderedDict([('id', 9),
                         ('latitude', home_lat + -1 * dlat),
                         ('longitude', home_long + 1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 10
            OrderedDict([('id', 10),
                         ('latitude', home_lat + -2 * dlat),
                         ('longitude', home_long + 1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 11
            OrderedDict([('id', 11),
                         ('latitude', home_lat + 2 * dlat),
                         ('longitude', home_long + 0 * dlong),
                         ('altitude', altitude)]),
            # cell no. 12
            OrderedDict([('id', 12),
                         ('latitude', home_lat + 1 * dlat),
                         ('longitude', home_long + 0 * dlong),
                         ('altitude', altitude)]),
            # cell no. 13
            OrderedDict([('id', 13),
                         ('latitude', home_lat + 0 * dlat),
                         ('longitude', home_long + 0 * dlong),
                         ('altitude', altitude)]),
            # cell no. 14
            OrderedDict([('id', 14),
                         ('latitude', home_lat + -1 * dlat),
                         ('longitude', home_long + 0 * dlong),
                         ('altitude', altitude)]),
            # cell no. 15
            OrderedDict([('id', 15),
                         ('latitude', home_lat + -2 * dlat),
                         ('longitude', home_long + 0 * dlong),
                         ('altitude', altitude)]),
            # cell no. 16
            OrderedDict([('id', 16),
                         ('latitude', home_lat + 2 * dlat),
                         ('longitude', home_long + -1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 17
            OrderedDict([('id', 17),
                         ('latitude', home_lat + 1 * dlat),
                         ('longitude', home_long + -1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 18
            OrderedDict([('id', 18),
                         ('latitude', home_lat + 0 * dlat),
                         ('longitude', home_long + -1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 19
            OrderedDict([('id', 19),
                         ('latitude', home_lat + -1 * dlat),
                         ('longitude', home_long + -1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 20
            OrderedDict([('id', 20),
                         ('latitude', home_lat + -2 * dlat),
                         ('longitude', home_long + -1 * dlong),
                         ('altitude', altitude)]),
            # cell no. 21
            OrderedDict([('id', 21),
                         ('latitude', home_lat + 2 * dlat),
                         ('longitude', home_long + -2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 22
            OrderedDict([('id', 22),
                         ('latitude', home_lat + 1 * dlat),
                         ('longitude', home_long + -2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 23
            OrderedDict([('id', 23),
                         ('latitude', home_lat + 0 * dlat),
                         ('longitude', home_long + -2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 24
            OrderedDict([('id', 24),
                         ('latitude', home_lat + -1 * dlat),
                         ('longitude', home_long + -2 * dlong),
                         ('altitude', altitude)]),
            # cell no. 25
            OrderedDict([('id', 25),
                         ('latitude', home_lat + -2 * dlat),
                         ('longitude', home_long + -2 * dlong),
                         ('altitude', altitude)]),
            ]
        
    def __len__(self):
        return len(self.Move)

## Streaming

In [13]:
class Streaming(threading.Thread):
    def __init__(self, drone, num_targets):
        self.drone = drone
        self.frame_queue = queue.Queue()
        self.flush_queue_lock = threading.Lock()
        self.frame_num = 0 
        self.renderer = None
        
        self.is_detecting = False
        self.num_targets = num_targets
        self.detected_targets = np.zeros(self.num_targets, dtype=bool)
        
        super().__init__()
        super().start()


    def start(self):
        self.drone.streaming.set_output_files(
            video=os.path.join(log_dir, timestr + ".mp4"),
        )
        
        # Setup your callback functions to do some live video processing
        self.drone.streaming.set_callbacks(
            raw_cb=self.yuv_frame_cb,
            h264_cb=self.h264_frame_cb,
            start_cb=self.start_cb,
            end_cb=self.end_cb,
            flush_raw_cb=self.flush_cb,
        )
        # Start video streaming
        self.drone.streaming.start()
        #self.renderer = PdrawRenderer(pdraw=self.drone.streaming)

    def stop(self):
        if self.renderer is not None:
            self.renderer.stop()
        # Properly stop the video stream and disconnect
        self.drone.streaming.stop()

    def yuv_frame_cb(self, yuv_frame):
        """
        This function will be called by Olympe for each decoded YUV frame.
            :type yuv_frame: olympe.VideoFrame
        """
        yuv_frame.ref()
        self.frame_queue.put_nowait(yuv_frame)

    def flush_cb(self, stream):
        if stream["vdef_format"] != olympe.VDEF_I420:
            return True
        with self.flush_queue_lock:
            while not self.frame_queue.empty():
                self.frame_queue.get_nowait().unref()
        return True

    def start_cb(self):
        pass

    def end_cb(self):
        pass

    def h264_frame_cb(self, h264_frame):
        pass

    def display_frame(self, yuv_frame):
        # the VideoFrame.info() dictionary contains some useful information
        # such as the video resolution
        info = yuv_frame.info()

        height, width = (  # noqa
            info["raw"]["frame"]["info"]["height"],
            info["raw"]["frame"]["info"]["width"],
        )

        # yuv_frame.vmeta() returns a dictionary that contains additional
        # metadata from the drone (GPS coordinates, battery percentage, ...)
        # convert pdraw YUV flag to OpenCV YUV flag
        cv2_cvt_color_flag = {
            olympe.VDEF_I420: cv2.COLOR_YUV2BGR_I420,
            olympe.VDEF_NV12: cv2.COLOR_YUV2BGR_NV12,
        }[yuv_frame.format()]

        # yuv_frame.as_ndarray() is a 2D numpy array with the proper "shape"
        # i.e (3 * height / 2, width) because it's a YUV I420 or NV12 frame

        # Use OpenCV to convert the yuv frame to RGB
        cv2frame = cv2.cvtColor(yuv_frame.as_ndarray(), cv2_cvt_color_flag)
#         cv2.imshow("Frames via Olympe", cv2frame)
#         cv2.waitKey(1)

        if not self.is_detecting:
            self.detected_targets = np.zeros(self.num_targets, dtype=bool)
        else: 
            print("detecting.....")
            for result in pyzbar.decode(cv2frame):
                idx = int(result.data) - 1
                try:
                    self.detected_targets[idx] = True
                except (ValueError, IndexError):
                    pass

    def run(self):
        main_thread = next(
            filter(lambda t: t.name == "MainThread", threading.enumerate())
        )
        while main_thread.is_alive():
            with self.flush_queue_lock:
                try:
                    yuv_frame = self.frame_queue.get(timeout=0.01)
                except queue.Empty:
                    continue
                try:
                    self.display_frame(yuv_frame)
                except Exception as e:
                    print(e)
                finally:
                    # Don't forget to unref the yuv frame. We don't want to
                    # starve the video buffer pool
                    yuv_frame.unref()

## Drone

In [14]:
class Drone:
    def __init__(self, drone_ip, num_targets, max_timestep, is_training=False):
        self.drone = olympe.Drone(drone_ip)
        self.drone.connect(retry=3)
        
        self.drone(GPSFixStateChanged(_policy = 'wait'))
        self._takeoff()
        if not is_training: self._setup_camera()

        self.streamer = Streaming(self.drone, num_targets)
        self.action = Action(self.drone)
        
        self.streamer.start()
        
        self.is_training = is_training
        self.num_targets = num_targets
        self.max_timestep = max_timestep
        self.timestep = 0
        self.visited_targets = np.zeros(self.num_targets, dtype=bool)
        self.target_positions = np.zeros(self.num_targets, dtype=np.uint8)
    
    def take_action(self, action):
        gps_reading = self.drone.get_state(GpsLocationChanged)
        battery = self.drone.get_state(BatteryStateChanged)["percent"]
        
        old_cell, new_cell, action_name = self.action.take_action(action)
        self.timestep += 1
        
        Simulation.cease_targets()
        time.sleep(0.1)
        detected_targets = self._detect_targets(new_cell)
        
        reward = self._get_reward(detected_targets) # !!! _get_reward must come before self.visited_targets is changed in _get_state
        state = self._get_state(new_cell, detected_targets) 
        if self.timestep >= self.max_timestep or np.all(self.visited_targets):
            done = True
            self.visited_targets[:] = False
        else:
            done = False
        info = {
            "action": str(action_name), 
            "direction": "Cell " + str(old_cell) + " --> " + "Cell " + str(new_cell),
            "positions": self.target_positions
        }
        
        logger.info(
            f"{gps_reading['latitude']},{gps_reading['longitude']},{gps_reading['altitude']},"
            f"{action_name},{old_cell},{new_cell},{battery}"
        ) 
        return state, reward, done, info

    def reset(self):
        old_cell, new_cell = self.action.reset()
        detected_targets = self._detect_targets(new_cell)
        self.timestep = 0
        Simulation.move_targets()
        return self._get_state(new_cell, detected_targets)
    
    def _get_state(self, new_cell, detected_targets):
        # {t, cell_id, [I1, I2, I3, ..., In]}
        
        self.visited_targets[detected_targets] = True
        
        return np.concatenate(([self.timestep, new_cell], self.visited_targets)).astype(np.uint8)
    
    def _get_reward(self, detected_targets):
        reward_scale = 1.5
        num_new_targets = np.count_nonzero(
            detected_targets & (detected_targets != self.visited_targets)
        )
        return reward_scale * num_new_targets if num_new_targets > 0 else -1
        
    def _detect_targets(self, cell_id):
        self.streamer.is_detecting = True
        time.sleep(1)
        print("detected from streamer: ", self.streamer.detected_targets)
        detected_targets = self.streamer.detected_targets
        self.streamer.is_detecting = False
#         if self.is_training:
#             self.update_moving_target_positions()
#             return self.target_positions == cell_id
        
#         detected_targets = np.zeros(self.num_targets, dtype=bool)

#         img = self._take_photo()
#         if img is None:
#             return detected_targets
        
#         for result in pyzbar.decode(img):
#             idx = int(result.data) - 1
#             try:
#                 detected_targets[idx] = True
#             except (ValueError, IndexError):
#                 pass
                
        return detected_targets

    def update_moving_target_positions(self):
        Simulation.update_moving_target_positions(self.target_positions)
        
    def _setup_camera(self):
#         assert self.drone.media_autoconnect
#         self.drone.media.integrity_check = True
#         is_indexed = False
#         while not is_indexed:
#             is_indexed = self.drone.media(
#                 indexing_state(state="indexed")
#             ).wait(_timeout=5).success()
        
#         self.drone(set_camera_mode(cam_id=0, value="photo")).wait()

#         assert self.drone(
#             set_photo_mode(
#                 cam_id=0,
#                 mode="single",
#                 format= "rectilinear",
#                 file_format="jpeg",
#                 # the following are ignored in photo single mode
#                 burst="burst_14_over_1s",
#                 bracketing="preset_1ev",
#                 capture_interval=5.0,
#             )
#         ).wait().success()

        assert self.drone(
            set_target(
                gimbal_id=0,
                control_mode="position",
                yaw_frame_of_reference="none",
                yaw=0.0,
                pitch_frame_of_reference="absolute",
                pitch=-90.0,
                roll_frame_of_reference="none",
                roll=0.0,
                )
            >> attitude(
                pitch_absolute=-90.0, _policy="wait", _float_tol=(1e-3, 1e-1)
                )
            ).wait(_timeout=20).success()
    
    def _take_photo(self):
        photo_saved = self.drone(photo_progress(result="photo_saved", _policy="wait"))
        self.drone(take_photo(cam_id=0)).wait()
        
        photo_taken = False
        tries = 0
        while not photo_taken:
            tries += 1
            if tries > 3:
#                 assert False, "take_photo timedout"
                print("take_photo timedout")
                return None
            photo_taken = photo_saved.wait(_timeout=5).success()
            
        # get the bytes of the image
        media_id = photo_saved.received_events().last().args["media_id"]
        for _ in range(5):
            media_info_response = requests.get(ANAFI_MEDIA_API_URL + media_id, timeout=10)
            if media_info_response.status_code == 200:
                break
        try:
            media_info_response.raise_for_status()
        except requests.exceptions.HTTPError as err:
            print(err)
            return None
        
        resource = media_info_response.json()["resources"][0]
        image_response = requests.get(ANAFI_URL + resource["url"], stream=True)
        image_response.raise_for_status()
        
        img = Image.open(BytesIO(image_response.content))
        
        # delete the image stored on the drone
        photo_deleted = False
        delete_tries = 0
        while not photo_deleted:
            delete_tries += 1
            if delete_tries > 3:
#                 assert False, "Failed to delete media {} {}".format(media_id, delete.explain())
                print("Failed to delete media {} {}".format(media_id, delete.explain()))
                break
            delete = delete_media(media_id, _timeout=10)
            photo_deleted = self.drone.media(delete).wait().success()
        
        return img
    
    def _takeoff(self):
        takeoff_success = self._success_if_takeoff()
        if not takeoff_success:
            print("Retrying taking off...")
            takeoff_success = self._success_if_takeoff()
    
    def _success_if_takeoff(self):
        return self.drone(
                FlyingStateChanged(state="hovering")
                | (TakeOff() & FlyingStateChanged(state="hovering"))
            ).wait(10).success()
        
    def _land(self):
        self.drone(PCMD(1, 0, 0, 0, 0, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
        assert self.drone(Landing() >> FlyingStateChanged(state="landed")).wait().success()
        
    def __del__(self):
        self.streamer.stop()
        self._land()
        self.drone.disconnect()
        del self.streamer
        del state
        del reward
        del action

## Simulation

In [15]:
class Simulation:
    sphinx = jsonrpclib.Server('http://127.0.0.1:8383')
    
    def __init__(self):
        pass
    
    @staticmethod
    def disable_battery():
        Simulation.sphinx.SetParam(machine='anafi4k',
                                   object='lipobattery/lipobattery',
                                   parameter='discharge_speed_factor',
                                   value='0')
    
    @staticmethod
    def reset_world():
        Simulation.sphinx.TriggerAction(machine='world',
                                        object='fwman/fwman',
                                        action='world_reset_all')
    
    def find_distance(x, y):
        x = x - y[:,np.newaxis]
        x = x**2
        x = x.sum(axis=2)
        x = np.sqrt(x)
        return x

    # check if the target overlaps with another target or border
    @classmethod
    def is_overlapping(cls, x, y=None, limit=0.4):
        # check with the borders
        borders_parallel_y = np.array([5, 3, 1, -1, -3, -5])
        x1 = abs(x[:,0] - borders_parallel_y[:,np.newaxis]) < 0.2 # what should this be changed to?

        borders_parallel_x = np.array([8, 4.8, 1.6, -1.6, -4.8, -8])
        x2 = abs(x[:,1] - borders_parallel_x[:,np.newaxis]) < 0.2 # what should this be changed to?
        
        # check if outside the boundaries
        x3 = (
            (x[:, 0] > borders_parallel_y.max()) | (x[:, 0] < borders_parallel_y.min()) | 
            (x[:, 1] > borders_parallel_x.max()) | (x[:, 1] < borders_parallel_x.min())
        )

        # check with another target
        if y is None: y = x
        x = cls.find_distance(x, y) < limit
        x = np.triu(x, k=1)

        x = np.vstack((x, x1, x2, x3))
        x = np.any(x, axis=0)
        return x
    
    def coord_to_cellid(locs):
        cell_boundaries = [
           (locs[:,0] > 3) & (locs[:,1] > 4.8),
           (locs[:,0] > 3) & (locs[:,1] > 1.6),
           (locs[:,0] > 3) & (locs[:,1] > -1.6),
           (locs[:,0] > 3) & (locs[:,1] > -4.8),
           (locs[:,0] > 3) & (locs[:,1] > -8),
           (locs[:,0] > 1) & (locs[:,1] > 4.8),
           (locs[:,0] > 1) & (locs[:,1] > 1.6),
           (locs[:,0] > 1) & (locs[:,1] > -1.6),
           (locs[:,0] > 1) & (locs[:,1] > -4.8),
           (locs[:,0] > 1) & (locs[:,1] > -8),
           (locs[:,0] > -1) & (locs[:,1] > 4.8),
           (locs[:,0] > -1) & (locs[:,1] > 1.6),
           (locs[:,0] > -1) & (locs[:,1] > -1.6),
           (locs[:,0] > -1) & (locs[:,1] > -4.8),
           (locs[:,0] > -1) & (locs[:,1] > -8),
           (locs[:,0] > -3) & (locs[:,1] > 4.8),
           (locs[:,0] > -3) & (locs[:,1] > 1.6),
           (locs[:,0] > -3) & (locs[:,1] > -1.6),
           (locs[:,0] > -3) & (locs[:,1] > -4.8),
           (locs[:,0] > -3) & (locs[:,1] > -8),
           (locs[:,0] > -5) & (locs[:,1] > 4.8),
           (locs[:,0] > -5) & (locs[:,1] > 1.6),
           (locs[:,0] > -5) & (locs[:,1] > -1.6),
           (locs[:,0] > -5) & (locs[:,1] > -4.8),
           (locs[:,0] > -5) & (locs[:,1] > -8),
        ]
        cell_ids = np.arange(25) + 1
        return np.select(cell_boundaries, cell_ids)
    
    @classmethod
    def gen_targets_pos(cls, num_targets):

        num_mobile = 3
        num_fixed = num_targets-num_mobile

        mu_x_fixed = 2.2
        variance_x_fixed = 1
        mu_y_fixed = -4 
        variance_y_fixed = 1.5
        distribution_fixed = multivariate_normal(
            [mu_x_fixed, mu_y_fixed], 
            [[variance_x_fixed, 0], [0, variance_y_fixed]]
        )

        mu_x_mobile = -3
        variance_x_mobile = 0.5
        mu_y_mobile = 2
        variance_y_mobile = 4.5
        distribution_mobile = multivariate_normal(
            [mu_x_mobile, mu_y_mobile],
            [[variance_x_mobile, 0], [0, variance_y_mobile]]
        )

        locs = np.empty((num_targets,2))
        locs[:num_fixed] = distribution_fixed.rvs(size=num_fixed)
        locs[num_fixed:] = distribution_mobile.rvs(size=num_mobile)

        overlapping_targets = Simulation.is_overlapping(locs)
        while np.any(overlapping_targets):
            num_fixed_overlaps = np.count_nonzero(overlapping_targets[:num_fixed])
            num_mobile_overlaps = np.count_nonzero(overlapping_targets[num_fixed:])
            locs[:num_fixed][overlapping_targets[:num_fixed]] = distribution_fixed.rvs(size=num_fixed_overlaps)
            locs[num_fixed:][overlapping_targets[num_fixed:]] = distribution_mobile.rvs(size=num_mobile_overlaps)
            overlapping_targets = Simulation.is_overlapping(locs)
        
        np.savetxt('../comm/positions.csv', 
                   np.hstack((np.arange(len(locs))[:,np.newaxis] + 1, locs)),
                   fmt='%.4f',
                   delimiter=',',
                  )
        
        print(locs)
        return cls.coord_to_cellid(locs) # , locs
    
    @classmethod
    def update_moving_target_positions(cls, arr, ids=[8, 9, 10]):
        ids = np.array(ids)
        for i in ids:
            locs = np.loadtxt("../comm/position_" + str(i) + ".csv", delimiter=',')
            arr[i-1] = cls.coord_to_cellid(locs[np.newaxis,:])
            
        return arr
    
    @staticmethod
    def cease_targets():
        f = open("../comm/toggle_movement.txt", "w")
        f.write("0")
        f.close()
    
    @staticmethod
    def move_targets():
        f = open("../comm/toggle_movement.txt", "w")
        f.write("1")
        f.close()
        
    @staticmethod
    def reset_targets(release=False):
        f = open("../comm/reset_position.txt", "w")
        if not release:
            f.write("1\n")
        else:
            f.write("0\n")
        f.close()


# Define a Gym Environment

In [16]:
class AnafiEnv(Env):
    def __init__(self, num_targets, max_timestep, drone_ip="10.202.0.1", is_training=False):
        super(AnafiEnv, self).__init__()
        
        Simulation.disable_battery()
        Simulation.cease_targets()
        
        self.num_targets = num_targets
        self.max_timestep = max_timestep
        self.begin(num_targets, max_timestep, is_training, drone_ip)
        
        self.action_space = spaces.Discrete(len(self.agent.action))
        self.observation_space = spaces.Box( # {t, cell_id, [I1, I2, I3, ..., In]}
            low=np.array([0, 1] + num_targets*[0]), 
            high=np.array([max_timestep, 25] + num_targets*[1]), 
            dtype=np.uint8,
        )
        
        Simulation.move_targets()
    
    def begin(self, num_targets, max_timestep, is_training, drone_ip="10.202.0.1"):
        self.agent = Drone(drone_ip, num_targets, max_timestep, is_training)
    
    def step(self, action):
        obs, reward, done, info = self.agent.take_action(action)
        Simulation.move_targets()
        return obs, reward, done, info
    
    def reset(self):
        Simulation.cease_targets()
        self.agent.target_positions = Simulation.gen_targets_pos(self.num_targets)
        Simulation.reset_targets()
        time.sleep(0.1)
        Simulation.reset_targets(release=True)
        return self.agent.reset()
    
    def render(self, mode='human'):
        pass
    
    def close(self):
        Simulation.cease_targets()
        del self.agent

# Run the simulation

In [17]:
def disp_info(action, observation, reward, done, info):
#     clear_output(wait=True)
    print("Action:", info["action"] + ",", info["direction"])
    print("Current positions:", info["positions"])
    print("State:", observation)
    print("Reward:", reward)
#     down_scale = 3
#     display(img.resize((img.size[0]//down_scale, img.size[1]//down_scale)))

In [18]:
try:
    del env
except Exception as e:
    pass

env = AnafiEnv(num_targets=10, max_timestep=20, is_training=False)
model = PPO.load("logs/7_run", env, verbose=1)
observation = env.reset()
# actions = [7, 4, 7, 3, 1, 2, 8, 4, 1, 5, 6]
for i in range(1000):
#     action = env.action_space.sample()
#     action = actions[i % len(actions)]
    action, _state = model.predict(observation, deterministic=True)
    observation, reward, done, info = env.step(action)
    disp_info(action, observation, reward, done, info)

    if done:
        print("The episode has ended. Resetting environment...")
        observation = env.reset()
        disp_info(action, observation, 0, done, info)
        break

env.close()

541 Server Error: Media Not Yet Indexed for url: http://10.202.0.1:80/api/v1/media/medias
Media are not yet indexed
2022-04-03 16:42:56,180 [ERROR] 	ulog - pdraw_sink_coded_video - VideoDecoder#7: coded video media format H264/AVCC not supported
2022-04-03 16:42:56,186 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,190 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,213 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,216 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,250 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
[[ 2.3828997  -4.11351234]
 [ 0.703049   -3.62159857]
 [ 4.65787633 -1.8335223 ]
 [ 0.45224124 -2.52548959]
 [ 3.27552794 -3.83261307]
 [ 2.13962266 -2.82732029]
 [ 1.56310399 -3.43751056]
 [-2.62085984  5.06571325]
 [-3.23997504  0.71824835]
 [-2.73295123  2.33623966]]


2022-04-03 16:42:56,292 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,321 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,325 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,357 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,361 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,394 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:56,397 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_

2022-04-03 16:42:57,204 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:57,208 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:57,242 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:57,245 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:57,275 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:57,277 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:42:57,311 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_

detecting.....
detecting.....
detecting.....
detecting.....
detecting.....
detecting.....


...

2022-04-03 16:43:55,690 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,693 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,723 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,725 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,759 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,762 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,795 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_

detecting.....
detecting.....
detecting.....
detecting.....
detecting.....
detecting.....


2022-04-03 16:43:55,900 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,903 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,936 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,939 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,969 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:55,971 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_frame_proto_get_buffer returned -71
2022-04-03 16:43:56,007 [ERROR] 	ulog - vmeta - vmeta_frame_proto_get_buffer:408:  err=71(Protocol error)
vmeta_

detecting.....
detecting.....
detecting.....
detecting.....
detecting.....
detected from streamer:  [False False False False False False False False False False]
Action: L, Cell 2 --> Cell 1
Current positions: [ 9 14  4 14  4  9  9 16 23 17]
State: [ 0 13  0  0  0  0  0  0  0  0  0  0]
Reward: 0


Exception ignored in: <function Drone.__del__ at 0x7f4105cc3f70>
Traceback (most recent call last):
  File "/tmp/ipykernel_5359/1962952474.py", line 208, in __del__
UnboundLocalError: local variable 'state' referenced before assignment


# Train the agent

In [ ]:
class SaveOnBestTrainingRewardCallback(BaseCallback):
    """
    Callback for saving a model (the check is done every ``check_freq`` steps)
    based on the training reward (in practice, we recommend using ``EvalCallback``).

    :param check_freq:
    :param log_dir: Path to the folder where the model will be saved.
      It must contains the file created by the ``Monitor`` wrapper.
    :param verbose: Verbosity level.
    """
    def __init__(self, check_freq: int, log_dir: str, verbose: int = 1):
        super(SaveOnBestTrainingRewardCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.log_dir = log_dir
        self.save_path = os.path.join(log_dir, 'best_model')
        self.best_mean_reward = -np.inf
        self.progress_dir = os.path.join(log_dir, 'progress')
        os.makedirs(self.progress_dir, exist_ok=True)

    def _init_callback(self) -> None:
        pass
#         # Create folder if needed
#         if self.save_path is not None:
#             os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % 5000 == 0:
            self.model.save(os.path.join(progress_dir, 'model_' + str(self.n_calls)))
            
        if self.n_calls % self.check_freq == 0:

            # Retrieve training reward
            x, y = ts2xy(load_results(self.log_dir), 'timesteps')
            if len(x) > 0:
                # Mean training reward over the last 100 episodes
                mean_reward = np.mean(y[-100:])
                if self.verbose > 0:
                    print(f"Num timesteps: {self.num_timesteps}")
                    print(f"Best mean reward: {self.best_mean_reward:.2f} - Last mean reward per episode: {mean_reward:.2f}")

                # New best model, you could save the agent here
                if mean_reward > self.best_mean_reward:
                    self.best_mean_reward = mean_reward
                    # Example for saving best model
                    if self.verbose > 0:
                        print(f"Saving new best model to {self.save_path}")
                    self.model.save(self.save_path)
        
        return True

In [ ]:
try:
    del env
    del model
except Exception as e:
    pass

# Create log dir
log_dir = "logs/"
os.makedirs(log_dir, exist_ok=True)
run = len([filename for filename in os.listdir(log_dir) if filename.endswith("monitor.csv")]) + 1
monitor_file = os.path.join(log_dir, str(run))

env = AnafiEnv(num_targets=10, max_timestep=20, is_training=True)
env = Monitor(env, monitor_file)
env = DummyVecEnv([lambda: env])

callback = SaveOnBestTrainingRewardCallback(check_freq=1024, log_dir=log_dir)

model = PPO("MlpPolicy", env, n_steps=2048, verbose=1, tensorboard_log=log_dir)
# model = PPO.load(os.path.join(log_dir, str(run-1) + "_run"), env)
model.learn(total_timesteps=524_288, callback=callback, tb_log_name="PPO_" + str(run), reset_num_timesteps=False)
model.save(os.path.join(log_dir, str(run) + "_run"))

# Test the agent

In [ ]:
try:
    del env
    del model
except Exception as e:
    pass

env = AnafiEnv(num_targets=10, max_timestep=15)
model = PPO.load("logs/7_run", env, verbose=1)

# header = ["timestep", "action"]

# episode = 9
# timestep = 0
# file = open('run_' + str(episode+1) + '.csv', 'w', encoding='UTF8', newline='')
# writer = csv.writer(file)
# writer.writerow(header)

obs = env.reset()

for i in range(1000):
    action, _state = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
#     timestep += 1
    
#     writer.writerow([str(timestep), info["action"]])

    if done:
        obs = env.reset()
        reset_target_pos()
        
#         if episode >= 10:
#             file.close()
#             break
#         episode += 1
#         timestep = 0

#         file = open('run_' + str(episode+1) + '.csv', 'w', encoding='UTF8', newline='')
#         writer = csv.writer(file)
#         writer.writerow(header)